## 背景

这篇文章我们将使用softmax分类器一起来构建一个简单的图像分类神经网络，其准确率可以达到32%。Softmax分类器是二元逻辑回归泛化到多元的情况。Softmax分类器会输出对应类别的概率。

我们会先定义一个softmax分类器，然后使用[cifar10](https://www.cs.toronto.edu/~kriz/cifar.html)的训练集来训练这个神经网络。最后使用测试集来测试神经网络的准确率。

我们开始吧。

## 引入依赖

类似前一篇教程[GetStarted](https://thoughtworksinc.github.io/DeepLearning.scala/demo/GettingStarted.html)，我们需要引入DeepLearning.scala的各个类。

In [7]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:2.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0-RC7`
import $ivy.`org.rauschig:jarchivelib:0.5.0`

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning
import org.nd4j.linalg.api.ndarray.INDArray
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.{
  DifferentiableHList,
  DifferentiableINDArray,
  Layer,
  Symbolic
}
import com.thoughtworks.deeplearning.Layer.Batch
import com.thoughtworks.deeplearning.Symbolic.Layers.Identity
import com.thoughtworks.deeplearning.Symbolic._
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathMethods./
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.cpu.nativecpu.NDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.indexing.{INDArrayIndex, NDArrayIndex}
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4s.Implicits._
import shapeless._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq

import $plugin.$                                                                             


import $ivy.$                                                           

import $ivy.$                                                               

import $ivy.$                                                           

import $ivy.$                                                              

import $ivy.$                                                             

import $ivy.$                                                             

import $ivy.$                                                                 

import $ivy.$                                                                

import $ivy.$                               


import $ivy.$                                             


import java.io.{FileInputStream, InputStream}



import com.thoughtworks.deeplearning

import org.nd4j.linalg.api.ndarray.INDArray

import com.thoughtworks.deeplearning.Differentiabl

为了减少`jupyter-scala`输出的行数，避免页面输出太长，需要设置`pprintConfig`。

In [8]:
pprintConfig() = pprintConfig().copy(height = 2)

## 构建神经网络

### 编写softmax

为了使用softmax分类器(softmax分类器是softmax和一个全连接组合起来的神经网络)，我们需要先编写softmax函数,公式：![](https://www.zhihu.com/equation?tex=f_j%28z%29%3D%5Cfrac%7Be%5E%7Bz_j%7D%7D%7B%5Csum_ke%5E%7Bz_k%7D%7D)

In [9]:
def softmax(implicit scores: INDArray @Symbolic): INDArray @Symbolic = {
  val expScores = exp(scores)
  expScores / expScores.sum(1)
}

defined function softmax

### 设置学习率

全连接层需要设置学习率，学习率是Weight变化的快慢的直观描述，学习率设置的过小会导致loss下降的很慢，需要更长时间来训练，学习率设置的过大虽然刚开始下降很快但是会导致在接近最低点的时候在附近徘徊loss下降会非常慢。

In [10]:
implicit def optimizer: Optimizer = new LearningRate {
  def currentLearningRate() = 0.00001
}

defined function optimizer

### 组合神经网络

定义一个全连接层并初始化[Weight](https://github.com/ThoughtWorksInc/DeepLearning.scala/wiki/Getting-Started#231--weight-intialization)，Weight应该是一个N*NumberOfClasses的INDArray,每个图片对应每个分类都有一个评分。评分就是该图片对应每个分类的可能概率。

In [11]:
//CIFAR10中的图片共有10个分类(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10

def createMyNeuralNetwork(implicit input: INDArray @Symbolic): INDArray @Symbolic = {
  val initialValueOfWeight = Nd4j.randn(3072, NumberOfClasses) * 0.001
  val weight: INDArray @Symbolic = initialValueOfWeight.toWeight
  val result: INDArray @Symbolic = input dot weight
  softmax.compose(result) //调用softmax方法，压缩结果值在0到1之间方便处理
}
val myNeuralNetwork = createMyNeuralNetwork

NumberOfClasses: Int = 10
defined function createMyNeuralNetwork
myNeuralNetwork: (Symbolic.To[INDArray]{type OutputData = org.nd4j.linalg.api.ndarray.INDArray;type OutputDelta = org.nd4j.linalg.api.ndarray.INDArray;type InputData = org.nd4j.linalg.api.ndarray.INDArray;type InputDelta = org.nd4j.linalg.api.ndarray.INDArray})#@ = Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum(Exp(Identity()),WrappedArray(1)))),Dot(Identity(),Weight([[0.00, -0.00, -0.00, 0.00, -0.00, -0.00, -0.00...

### 组合LossFunction

为了判断神经网络判断的结果好坏，我们需要编写损失函数Loss Function，这里我们使用cross-entropy loss将此次判断的结果和真实结果进行对比并返回评分，公式：
![](https://zhihu.com/equation?tex=%5Cdisplaystyle+H%28p%2Cq%29%3D-%5Csum_xp%28x%29+logq%28x%29)

In [12]:
def lossFunction(implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val input = pair.head
  val expectedOutput = pair.tail.head
  val probabilities = myNeuralNetwork.compose(input)

  -(expectedOutput * log(probabilities)).mean //此处上面的交叉熵损失公式对应
}

defined function lossFunction

## 准备数据

### 读取数据

为了从CIFAR10 database中读取训练数据和测试数据的图片和分类信息。我们需要[`import $file.ReadCIFAR10ToNDArray`](https://github.com/ThoughtWorksInc/DeepLearning.scala-website/blob/master/ipynbs/ReadCIFAR10ToNDArray.sc)。这是一个包含读取和处理CIFAR10 数据的脚本文件，由本教程提供。

In [13]:
import $file.ReadCIFAR10ToNDArray

//加载train数据,我们读取1000条数据作为训练数据
val trainNDArray =
  ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/data_batch_1.bin", 1000)

//加载测试数据，我们读取100条作为测试数据
val testNDArray =
  ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

Compiling ReadCIFAR10ToNDArray.sc


import $file.$                   

//加载train数据,我们读取1000条数据作为训练数据

trainNDArray: INDArray :: INDArray :: HNil = [[0.23, 0.17, 0.20, 0.27, 0.38, 0.46, 0.54, 0.57, 0.58, 0.58, 0.51, 0.49, 0.55, 0.56, 0.54, 0.50, 0.54, 0.52, 0.48, 0.54, 0.54, 0.52, 0.53, 0.54, 0.59, 0.64, 0....
testNDArray: INDArray :: INDArray :: HNil = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....

### 处理数据

为了方便softmax分类器处理数据，我们先处理标签数据([one hot encoding](https://en.wikipedia.org/wiki/One-hot))：将N行一列的NDArray转换为N行NumberOfClasses列的NDArray，每行对应的正确分类的值为1，其它列的值为0。这里区分训练集和测试集的原因是为了能看出网络是否被过度训练导致[过拟合](https://en.wikipedia.org/wiki/Overfitting)。处理标签数据的时候我们使用了[Utils](https://github.com/ThoughtWorksInc/DeepLearning.scala-website/blob/master/ipynbs/Utils.sc)，也由本教程提供。

In [14]:
val trainData = trainNDArray.head
val testData = testNDArray.head

val trainExpectResult = trainNDArray.tail.head
val testExpectResult = testNDArray.tail.head

import $file.Utils

val vectorizedTrainExpectResult = Utils.makeVectorized(trainExpectResult, NumberOfClasses)
val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

Compiling Utils.sc


trainData: INDArray = [[0.23, 0.17, 0.20, 0.27, 0.38, 0.46, 0.54, 0.57, 0.58, 0.58, 0.51, 0.49, 0.55, 0.56, 0.54, 0.50, 0.54, 0.52, 0.48, 0.54, 0.54, 0.52, 0.53, 0.54, 0.59, 0.64, 0....
testData: INDArray = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....
trainExpectResult: INDArray = [6.00, 9.00, 9.00, 4.00, 1.00, 1.00, 2.00, 7.00, 8.00, 3.00, 4.00, 7.00, 7.00, 2.00, 9.00, 9.00, 9.00, 3.00, 2.00, 6.00, 4.00, 3.00, 6.00, 6.00, 2.00, 6.00, 3.0...
testExpectResult: INDArray = [3.00, 8.00, 8.00, 0.00, 6.00, 6.00, 1.00, 6.00, 3.00, 1.00, 0.00, 9.00, 5.00, 7.00, 9.00, 8.00, 5.00, 7.00, 8.00, 6.00, 7.00, 0.00, 4.00, 9.00, 5.00, 2.00, 4.0...
import $file.$    


vectorizedTrainExpectResult: INDArray = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
...
vectorizedTestExpectResult: INDArray = [[0.00, 0.00

## 训练神经网络

为了观察神经网络训练的过程，我们需要输出`loss`，在训练神经网络时，loss的变化趋势应该是越来越低的

In [15]:
val lossSeq = for (iteration <- 0 until 2000) yield {
  val loss = lossFunction.train(trainData :: vectorizedTrainExpectResult :: HNil)
  if(iteration % 100 == 0){
    println(s"at iteration $iteration loss is $loss")
  }
  loss
}

val plot = Seq(
  Scatter(lossSeq.indices, lossSeq)
)

plot.plot(
  title = "loss by time"
)

at iteration 0 loss is 0.2304002685546875
at iteration 100 loss is 0.1909240234375
at iteration 200 loss is 0.17821024169921876
at iteration 300 loss is 0.170245556640625
at iteration 400 loss is 0.164287255859375
at iteration 500 loss is 0.1594460693359375
at iteration 600 loss is 0.1553224365234375
at iteration 700 loss is 0.151703857421875
at iteration 800 loss is 0.1484637939453125
at iteration 900 loss is 0.14552034912109374
at iteration 1000 loss is 0.1428169677734375
at iteration 1100 loss is 0.140312744140625
at iteration 1200 loss is 0.137976904296875
at iteration 1300 loss is 0.1357855224609375
at iteration 1400 loss is 0.1337197021484375
at iteration 1500 loss is 0.13176416015625
at iteration 1600 loss is 0.1299063232421875
at iteration 1700 loss is 0.12813583984375
at iteration 1800 loss is 0.1264439208984375
at iteration 1900 loss is 0.1248230712890625


lossSeq: IndexedSeq[cmd11Wrapper.<refinement>.this.type.OutputData] = Vector(
  0.2304002685546875,
...
plot: Seq[Scatter] = List(
  Scatter(
...
res14_2: String = "plot-922967534"

## 验证神经网络预测准确率

我们使用已经处理好的测试数据来验证神经网络的准确率。准确率应该在32%左右。

In [16]:
val right = Utils.getAccuracy(myNeuralNetwork.predict(testData), testExpectResult)
println(s"the result is $right %")

the result is 32.0 %


right: Double = 32.0

## 总结

在这节中我们学到了：

* 准备和处理CIFAR10数据
* 编写softmax分类器
* 使用softmax分类器编写的神经网络预测图片对应于每个分类的概率


[完整代码](https://github.com/izhangzhihao/deeplearning-tutorial/blob/master/src/main/scala/com/thoughtworks/deeplearning/tutorial/SoftmaxLinearClassifier.scala)